<a href="https://colab.research.google.com/github/monsegutier/GRUPO-6-PROC.-DE-DATOS/blob/main/src/crear_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pobreza en Argentina**

En este trabajo se analiza la pobreza en Argentina de acuerdo a los datos de la Encuesta Permanente de Hogares (EPH). Las bases de datos que se toman en este caso son las del último trimestre del año 2022 (últimas disponibles).

De acuerdo a los ingresos de cada hogar y los datos de la canasta básica que proporciona INDEC se puede estimar el porcentaje de pobreza en Argentina.

Para calcular el ingreso del hogar se estima primero el porcentaje correspondiente al adulto equivalente en cada miembro del hogar, según género y edad, siguiendo las equivalencias dispuestas por INDEC (ver Cuadro 4 del archivo canasta_01_23DF5760FF57.pdf)

En este caso, la línea de pobreza para un adulto equivalente fue de $49.358.

Las fuentes de los bases de datos son:


**Bases de datos**

https://www.indec.gob.ar/indec/web/Institucional-Indec-BasesDeDatos


**Diseño de registro y estructura para las bases**

https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_disenoreg_09.pdf


**Canasta básica alimentaria diciembre 2022**

https://www.indec.gob.ar/uploads/informesdeprensa/canasta_01_23DF5760FF57.pdf



# Importamos las librerías






In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import warnings

warnings.filterwarnings('ignore')

# 1- Recolección y preparación de datos:

## **Creamos el dataset**





## Carga de datos y creacion de Dataframes para trabajar

In [10]:
#cargamos el archivo de individuos "usu_individual_T422.csv"
individuos = 'https://raw.githubusercontent.com/monsegutier/GRUPO-6-PROC.-DE-DATOS/main/data/usu_individual_T422.csv'
# from google.colab import files
# uploaded  = files.upload()

In [11]:
#cargamos el archivo de hogares "usu_hogar_T422.csv"
hogares = 'https://raw.githubusercontent.com/monsegutier/GRUPO-6-PROC.-DE-DATOS/main/data/usu_hogar_T422.csv'
#from google.colab import files
#uploaded  = files.upload()

In [12]:
#creamos el dataframe individuos
#df_individual = pd.read_csv("usu_individual_T422.csv", sep=';')
df_individual = pd.read_csv(individuos, sep=';')

In [13]:
#creamos el dataframe hogares
#df_hogares = pd.read_csv("usu_hogar_T422.csv", sep=";")
df_hogares = pd.read_csv(hogares, sep=";")

## Dataframe checks

In [14]:
#medimos las dimensiones
print("Dimensiones de dataframe individuos:", df_individual.shape)
print("Dimensiones de dataframe hogares:", df_hogares.shape)


Dimensiones de dataframe individuos: (48545, 177)
Dimensiones de dataframe hogares: (16740, 88)


In [15]:
#vemos los primeros registros
print(df_individual.head())
print(df_hogares.head())

                          CODUSU  ANO4  TRIMESTRE  NRO_HOGAR  COMPONENTE  H15  \
0  TQRMNOPRUHKMLSCDEGJBF00788920  2022          4          1           2    1   
1  TQRMNORSSHMMMLCDEGJBF00714103  2022          4          1           1    1   
2  TQRMNORSSHMMMLCDEGJBF00714103  2022          4          1           2    1   
3  TQRMNORSSHMMMLCDEGJBF00714103  2022          4          1           3    1   
4  TQRMNORQUHLMLOCDEFPCH00715601  2022          4          1           1    1   

   REGION MAS_500  AGLOMERADO  PONDERA  ...  PDECIFR  ADECIFR      IPCF  \
0      43       S          13      464  ...      NaN       12      0.00   
1      43       S          13      532  ...      NaN        7  61333.33   
2      43       S          13      532  ...      NaN        7  61333.33   
3      43       S          13      532  ...      NaN        7  61333.33   
4      44       N           9      348  ...      9.0        7  76666.67   

   DECCFR  IDECCFR  RDECCFR  GDECCFR  PDECCFR  ADECCFR  PONDIH

## Empezamos a trabajar los datasets

Para integrarlos, es necesario calcular el porcentaje de adulto equivalente de cada miembro del hogar, según edad y género, en el dataframe individuos  y luego integrarlo con el dataframe hogares


In [16]:
# le asignamos un % a cada persona que vive en el hogar según género y edad, en la variable 'AE_ctrol_ind'
# ver Cuadro 4 de canasta_01_23DF5760FF57.pdf

condiciones = [
    (df_individual ['CH04'] >= 1) & (df_individual['CH06'] < 1),
    (df_individual['CH04'] >= 1) & (df_individual['CH06'] == 1),
    (df_individual['CH04'] >= 1) & (df_individual['CH06'] == 2),
    (df_individual['CH04'] >= 1) & (df_individual['CH06'] == 3),
    (df_individual['CH04'] >= 1) & (df_individual['CH06'] == 4),
    (df_individual['CH04'] >= 1 ) & ( df_individual['CH06'] == 5),
    (df_individual['CH04'] >= 1) & (df_individual['CH06'] == 6),
    (df_individual['CH04'] >= 1) & (df_individual['CH06'] == 7),
    (df_individual['CH04'] >= 1) & (df_individual['CH06'] == 8),
    (df_individual['CH04'] >= 1) & (df_individual['CH06'] == 9),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] == 10),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] == 11),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] == 12),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] >= 13) & (df_individual['CH06'] <=14),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] >= 15) & (df_individual['CH06'] <=17),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] >= 18) & (df_individual['CH06'] <=29),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] >= 30) & (df_individual['CH06'] <=45),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] >= 46) & (df_individual['CH06'] <=60),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] >= 61) & (df_individual['CH06'] <=75),
    (df_individual['CH04'] == 2) & (df_individual['CH06'] > 75),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] ==10),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] ==11),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] ==12),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] ==13),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] ==14),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] ==15),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] ==16),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] ==17),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] >= 18) & (df_individual['CH06'] <=29),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] >= 30) & (df_individual['CH06'] <=45),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] >= 46) & (df_individual['CH06'] <=60),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] >= 61) & (df_individual['CH06'] <=75),
    (df_individual['CH04'] == 1) & (df_individual['CH06'] > 75)
    ]

opciones = [0.35,0.37,0.46,0.51,0.55,0.60,0.64,0.66,0.68,0.69,0.70,0.72,0.74,0.76,0.77,0.76,0.77,0.76,0.67,0.63,0.79,0.82,0.85,0.90,
            0.96,1.00,1.03,1.04,1.02,1.00,1.00,0.83,0.74]



In [17]:
df_individual['AE_ctrol_ind']=np.select(condiciones,opciones)


In [21]:
df_individual.head(5)

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH,AE_ctrol_ind
0,TQRMNOPRUHKMLSCDEGJBF00788920,2022,4,1,2,1,43,S,13,464,...,12,0.00,12,12.0,12,12.0,NaN,12,0,1.02
1,TQRMNORSSHMMMLCDEGJBF00714103,2022,4,1,1,1,43,S,13,532,...,7,61333.33,6,7.0,6,6.0,NaN,6,635,0.83
2,TQRMNORSSHMMMLCDEGJBF00714103,2022,4,1,2,1,43,S,13,532,...,7,61333.33,6,7.0,6,6.0,NaN,6,635,0.67
3,TQRMNORSSHMMMLCDEGJBF00714103,2022,4,1,3,1,43,S,13,532,...,7,61333.33,6,7.0,6,6.0,NaN,6,635,1.00
4,TQRMNORQUHLMLOCDEFPCH00715601,2022,4,1,1,1,44,N,9,348,...,7,76666.67,7,8.0,6,NaN,8.0,6,341,1.00


In [22]:
#seleccionamos solo las columnas que nos interesan para este estudio (ver archivo "EPH_disenoreg_09 disponible en https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_disenoreg_09.pdf")

df_personas = df_individual[['CODUSU','NRO_HOGAR','CH03','CH04','CH06','CH08','NIVEL_ED','CAT_OCUP','CAT_INAC','AE_ctrol_ind']]

In [27]:
df_personas.head(5)

,CODUSU,NRO_HOGAR,CH03,CH04,CH06,CH08,NIVEL_ED,CAT_OCUP,CAT_INAC,AE_ctrol_ind
0,TQRMNOPRUHKMLSCDEGJBF00788920,1,2,1,22,2,4,3,0,1.02
1,TQRMNORSSHMMMLCDEGJBF00714103,1,1,1,73,1,5,0,1,0.83
2,TQRMNORSSHMMMLCDEGJBF00714103,1,2,2,69,1,5,0,1,0.67
3,TQRMNORSSHMMMLCDEGJBF00714103,1,3,1,40,3,1,3,0,1.00
4,TQRMNORQUHLMLOCDEFPCH00715601,1,1,1,58,1,1,3,0,1.00


In [29]:
#sumamos el % de cada persona que compone el hogar (las variables CODUSU y NRO_HOGAR identifican a cada hogar)

df_suma = df_personas.groupby(['CODUSU','NRO_HOGAR']).AE_ctrol_ind.sum()

In [30]:
df_suma.head(5)

CODUSU                         NRO_HOGAR
TQRMNOPPQHJKLLCDEFKID00798130  1            1.50
TQRMNOPPQHJLKUCDEFKID00796257  1            2.00
TQRMNOPPQHJMLOCDEHPJB00795718  1            3.79
TQRMNOPPQHJNLPCDEHJGH00793308  1            4.43
TQRMNOPPQHJNLTCDEFKID00798140  1            2.02
Name: AE_ctrol_ind, dtype: float64

In [31]:
df_suma.shape

(16740,)

In [32]:
#unimos los dos dataframes según las variables CODUSU y NRO_HOGAR

df_personas_suma = pd.merge(df_personas, df_suma, on=['CODUSU','NRO_HOGAR'])

In [33]:
df_personas_suma.head(5)

,CODUSU,NRO_HOGAR,CH03,CH04,CH06,CH08,NIVEL_ED,CAT_OCUP,CAT_INAC,AE_ctrol_ind_x,AE_ctrol_ind_y
0,TQRMNOPRUHKMLSCDEGJBF00788920,1,2,1,22,2,4,3,0,1.02,1.78
1,TQRMNOPRUHKMLSCDEGJBF00788920,1,1,2,24,2,6,3,0,0.76,1.78
2,TQRMNORSSHMMMLCDEGJBF00714103,1,1,1,73,1,5,0,1,0.83,2.50
3,TQRMNORSSHMMMLCDEGJBF00714103,1,2,2,69,1,5,0,1,0.67,2.50
4,TQRMNORSSHMMMLCDEGJBF00714103,1,3,1,40,3,1,3,0,1.00,2.50


In [34]:
#renombramos las columnas

df_personas_suma = df_personas_suma.rename(columns={
     'CH03':'parentesco',
     'CH04':'genero',
     'CH06':'edad',
     'CH08':'cob_medica',
     'AE_ctrol_ind_x':'AE_ctrol_ind',
     'AE_ctrol_ind_y':'AE_ctrol_ind_sum'})

In [35]:
df_personas_suma.head(2)

,CODUSU,NRO_HOGAR,parentesco,genero,edad,cob_medica,NIVEL_ED,CAT_OCUP,CAT_INAC,AE_ctrol_ind,AE_ctrol_ind_sum
0,TQRMNOPRUHKMLSCDEGJBF00788920,1,2,1,22,2,4,3,0,1.02,1.78
1,TQRMNOPRUHKMLSCDEGJBF00788920,1,1,2,24,2,6,3,0,0.76,1.78


In [36]:
df_personas_suma.shape

(48545, 11)

In [37]:
#seleccionamos solo los que cumplen la condición de jefes/as de hogar

df_personas_jefes = df_personas_suma.loc[df_personas_suma.loc[:, 'parentesco'] == 1]

In [38]:
df_personas_jefes.head(5)

,CODUSU,NRO_HOGAR,parentesco,genero,edad,cob_medica,NIVEL_ED,CAT_OCUP,CAT_INAC,AE_ctrol_ind,AE_ctrol_ind_sum
1,TQRMNOPRUHKMLSCDEGJBF00788920,1,1,2,24,2,6,3,0,0.76,1.78
2,TQRMNORSSHMMMLCDEGJBF00714103,1,1,1,73,1,5,0,1,0.83,2.50
5,TQRMNORQUHLMLOCDEFPCH00715601,1,1,1,58,1,1,3,0,1.00,2.78
8,TQRMNOQWXHLMNOCDEIJAH00719609,2,1,1,31,4,3,3,0,1.00,4.10
13,TQSMNORXSHKLMUCDEIJAH00794113,1,1,1,49,4,5,2,0,1.00,4.33


In [39]:
#comprobamos la cantidad de filas

df_personas_jefes.shape

(16740, 11)

In [40]:
#exportamos el dataframe como .csv
df_personas_jefes.to_csv('df_personas_jefes.csv', sep=';', index=False)

In [41]:
#comprobamos la cantidad de filas

df_hogares.shape

(16740, 88)

Vemos que la cantidad de filas coincide con la del dataframe df_personas_jefes


In [42]:
#unimos los dos dataframes, que tienen la misma cantidad de filas, según las variables CODUSU y NRO_HOGAR

df_dataset = pd.merge(df_hogares, df_personas_jefes, on=['CODUSU','NRO_HOGAR'])

In [43]:
#comprobamos la cantidad de filas

df_dataset.shape

(16740, 97)

In [44]:
df_dataset.head(2)

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,REALIZADA,REGION,MAS_500,AGLOMERADO,PONDERA,IV1,...,VII2_4,parentesco,genero,edad,cob_medica,NIVEL_ED,CAT_OCUP,CAT_INAC,AE_ctrol_ind,AE_ctrol_ind_sum
0,TQRMNOSPYHKOLQCDEGNFJ00789611,2022,4,1,1,44,N,17,356,1,...,0,1,1,46,4,3,2,0,1.00,3.15
1,TQRMNOQXUHMOLQCDEGNFJ00713295,2022,4,1,1,44,N,17,239,1,...,0,1,1,69,1,2,0,1,0.83,3.93


In [ ]:
#df_dataset = df_dataset.drop(['Unnamed: 0'], axis=1)

In [45]:
#la variable ITF representa el monto de ingreso total familiar (ver archivo "EPH_disenoreg_09 disponible en https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_disenoreg_09.pdf")
#a este monto lo dividimos por la suma de % de las personas que residen en el hogar para obtener el ingreso por adulto equivalente

df_dataset['adulto_equivalente']= df_dataset.apply(lambda row: row.ITF/row.AE_ctrol_ind_sum, axis=1)

In [46]:
df_dataset['adulto_equivalente']= round(df_dataset['adulto_equivalente'], 2)

In [47]:
df_dataset.head(5)

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,REALIZADA,REGION,MAS_500,AGLOMERADO,PONDERA,IV1,...,parentesco,genero,edad,cob_medica,NIVEL_ED,CAT_OCUP,CAT_INAC,AE_ctrol_ind,AE_ctrol_ind_sum,adulto_equivalente
0,TQRMNOSPYHKOLQCDEGNFJ00789611,2022,4,1,1,44,N,17,356,1,...,1,1,46,4,3,2,0,1.00,3.15,126984.13
1,TQRMNOQXUHMOLQCDEGNFJ00713295,2022,4,1,1,44,N,17,239,1,...,1,1,69,1,2,0,1,0.83,3.93,0.00
2,TQRMNORRSHJMKMCDEOJAH00794456,2022,4,1,1,44,N,93,40,2,...,1,2,49,1,6,3,0,0.76,3.82,0.00
3,TQRMNOSVUHLMKNCDEOJAH00720329,2022,4,1,1,44,N,93,95,1,...,1,2,76,1,3,0,1,0.63,0.63,146031.75
4,TQRMNOSQWHJOKNCDEOJAH00794457,2022,4,1,1,44,N,93,89,1,...,1,2,44,1,6,3,0,0.77,2.56,19531.25


In [48]:
#creamos la variable objetivo, pobreza

"""
De acuerdo con el archivo "canasta_01_23DF5760FF57" (disponible en https://www.indec.gob.ar/uploads/informesdeprensa/canasta_01_23DF5760FF57.pdf)
la canasta básica total para un adulto equivalente, para que no ingrese en la línea de pobreza, es de $49.6358.

Creamos la varibale pobreza, donde todos los hogares que tengan ingresos menores a este valor son pobres.
"""

df_dataset['pobreza'] = np.where(df_dataset['adulto_equivalente']<49358,1,0)


In [49]:
df_dataset['pobreza'].value_counts()

1    8549
0    8191
Name: pobreza, dtype: int64

In [50]:
df_dataset.head(5)

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,REALIZADA,REGION,MAS_500,AGLOMERADO,PONDERA,IV1,...,genero,edad,cob_medica,NIVEL_ED,CAT_OCUP,CAT_INAC,AE_ctrol_ind,AE_ctrol_ind_sum,adulto_equivalente,pobreza
0,TQRMNOSPYHKOLQCDEGNFJ00789611,2022,4,1,1,44,N,17,356,1,...,1,46,4,3,2,0,1.00,3.15,126984.13,0
1,TQRMNOQXUHMOLQCDEGNFJ00713295,2022,4,1,1,44,N,17,239,1,...,1,69,1,2,0,1,0.83,3.93,0.00,1
2,TQRMNORRSHJMKMCDEOJAH00794456,2022,4,1,1,44,N,93,40,2,...,2,49,1,6,3,0,0.76,3.82,0.00,1
3,TQRMNOSVUHLMKNCDEOJAH00720329,2022,4,1,1,44,N,93,95,1,...,2,76,1,3,0,1,0.63,0.63,146031.75,0
4,TQRMNOSQWHJOKNCDEOJAH00794457,2022,4,1,1,44,N,93,89,1,...,2,44,1,6,3,0,0.77,2.56,19531.25,1


In [51]:
df_dataset.to_csv('dataset.csv')

# 2 - Análisis exploratorio de datos:

# 3 - Procesamiento y análisis de datos:

# 4 - Visualización de datos:

# 5 - Comunicación efectiva de resultados: